In [0]:
from pyspark.sql import functions as F

In [0]:
"""
Calculate Derived Metrics
ride_duration: (ended_at - started_at) in minutes/seconds
ride_distance: Haversine distance between start/end coordinates
day_of_week: Extract from started_at
hour_of_day: Extract hour from started_at
is_weekend: Boolean flag
ride_month, ride_year: For partitioning
"""
bikeshare_silver_df = spark.read.table("`bikeshare-dc-data`.silver.dc_rideshare_st")
# need to extract the seconds property from the struct thats in the ride_duration column
bikeshare_silver_df = bikeshare_silver_df.withColumn("ride_duration (minutes)", F.timestamp_diff("MINUTE", F.col("started_at"), F.col("ended_at")))

# day of the week from started_at
bikeshare_silver_df = bikeshare_silver_df.withColumn("day_of_week", F.dayofweek(F.col("started_at")))
# hour of day
bikeshare_silver_df = bikeshare_silver_df.withColumn("hour_of_day", F.hour(F.col("started_at")))

# determine if day is weekedend
bikeshare_silver_df = bikeshare_silver_df.withColumn("is_weekend", F.when(F.col("day_of_week").isin([6,7]), True).otherwise(False))
# extract month and year from started_at
bikeshare_silver_df = bikeshare_silver_df.withColumn("ride_month", F.month(F.col("started_at")))
bikeshare_silver_df = bikeshare_silver_df.withColumn("ride_year", F.year(F.col("started_at")))

display(bikeshare_silver_df)

In [0]:
"""
Categorize Rides
trip_type: "round_trip" if start_station_id == end_station_id, else "one_way"
time_of_day: "morning", "afternoon", "evening", "night" based on hour
"""
# Just use PySpark functions - no UDF needed!
bikeshare_silver_df = bikeshare_silver_df.withColumn(
    "hour_of_day", F.hour("started_at").cast("int")
).withColumn(
    "trip_type", 
    F.when(F.col("start_station_id") == F.col("end_station_id"), "round_trip").otherwise("one_way")
).withColumn(
    "time_of_day",
    F.when((F.col("hour_of_day") >= 4) & (F.col("hour_of_day") < 12), "morning")
     .when((F.col("hour_of_day") >= 12) & (F.col("hour_of_day") < 17), "afternoon")
     .when((F.col("hour_of_day") >= 17) & (F.col("hour_of_day") < 21), "evening")
     .otherwise("night")
)
bikeshare_silver_df = bikeshare_silver_df.withColumn("trip_type", F.when(F.col("start_station_id") == F.col("end_station_id"), "round_trip").otherwise("one_way"))
bikeshare_silver_df = bikeshare_silver_df.withColumn("hour_of_day", F.col("hour_of_day").cast("int"))

In [0]:
bikeshare_silver_df.printSchema()

In [0]:
display(bikeshare_silver_df)

# Data Validation & Filtering

In [0]:
from math import radians

In [0]:
# rename ride_duration column
bikeshare_silver_df = bikeshare_silver_df.withColumnRenamed("ride_duration (minutes)", "ride_duration_minutes")

In [0]:
# Haversine distance calculation
bikeshare_silver_df = bikeshare_silver_df.withColumn(
    "ride_distance_km",
    F.round(F.acos(
        F.sin(F.radians(F.col("start_lat"))) * F.sin(F.radians(F.col("end_lat"))) +
        F.cos(F.radians(F.col("start_lat"))) * F.cos(F.radians(F.col("end_lat"))) *
        F.cos(F.radians(F.col("end_lng")) - F.radians(F.col("start_lng")))
    ) * 6371, 2)  # Earth's radius in kilometers
)

# For round trips, estimate based on duration (avg speed: 12 km/h)
bikeshare_silver_df = bikeshare_silver_df.withColumn(
    "ride_distance_km",
    F.when(
        (F.col("trip_type") == "round_trip") | (F.col("ride_distance_km") == 0),
        F.round((F.col("ride_duration_minutes") / 60) * 12, 2)
    ).otherwise(
        F.col("ride_distance_km")
    )
)

In [0]:
display(bikeshare_silver_df)

In [0]:
"""
Business Rule Validation
Filter out rides where ended_at <= started_at (invalid times)
Remove rides with duration < 1 minute or > 24 hours (likely errors)
Flag or remove rides with impossible distances (e.g., > 50 miles for bikeshare)
"""
bikeshare_silver_df = bikeshare_silver_df.filter(F.col("ended_at") > F.col("started_at"))
bikeshare_silver_df = bikeshare_silver_df.filter((F.col("ride_duration_minutes") >= 1) & (F.col("ride_duration_minutes") <= 1440))
bikeshare_silver_df = bikeshare_silver_df.filter((F.col("ride_distance_km") / 1.609) <= 50) # 1.609 is the conversion factor from km to miles


In [0]:
bikeshare_silver_df.count()

## Standardize values

In [0]:
# trim all columns
bikeshare_silver_df = bikeshare_silver_df.select(*[F.trim(F.col(c)).alias(c) for c in bikeshare_silver_df.columns])
bikeshare_silver_df = bikeshare_silver_df.withColumn("member_casual", F.lower(F.col("member_casual")))


## Schema Optimization

In [0]:
"""_ingestion_timestamp: When record was processed to silver
_source_file: Track which bronze file this came from
"""
bikeshare_silver_df = bikeshare_silver_df.withColumn("_ingestion_timestamp", F.current_timestamp())
bikeshare_silver_df = bikeshare_silver_df.withColumn("_source_file", F.col("_metadata.file_path"))


In [0]:
bikeshare_silver_df.count()

In [0]:
(
    bikeshare_silver_df.write
    .format("delta")
    .mode("overwrite")
    .partitionBy("rider_year")
)

In [0]:
import time# First, write without partitioning

start_time = time.time()

(bikeshare_silver_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("`bikeshare-dc-data`.bikeshare_schema.rides_silver_no_partition")
)

write_time_no_partition = time.time() - start_time
print(f"Write time (no partition): {write_time_no_partition:.2f} seconds")

In [0]:
start_time = time.time()

(bikeshare_silver_df
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("ride_year", "ride_month")
    .option("overwriteSchema", "true")
    .saveAsTable("`bikeshare-dc-data`.bikeshare_schema.rides_silver_partitioned")
)

write_time_partitioned = time.time() - start_time
print(f"Write time (partitioned): {write_time_partitioned:.2f} seconds")